In [16]:
import pickle
import itertools
import numpy as np
from scipy import spatial
from scipy.stats import norm
import nltk.data
from nltk import pos_tag
from nltk.corpus import wordnet as wn
from nltk.corpus import reuters
from nltk. corpus import gutenberg
from nltk.corpus import brown
from nltk.tokenize import sent_tokenize
from gensim.models import KeyedVectors
from keras.layers import Input, Dense, Lambda, Layer
from keras.callbacks import ModelCheckpoint
from keras.models import Model
from keras import backend as K
from keras import metrics

In [17]:
#Read Excel with Pandas
import pandas as pd
from pandas import ExcelFile
df = pd.read_excel('laufzeit_vertarg.xlsx',sheet_name = 'Sheet1')

In [18]:
df.columns

Index(['Beschreibung', 'Skript'], dtype='object')

In [19]:
df.head()

,Beschreibung,Skript
0,Maximize SAPGuiSession,"SAPGuiSession(""Session"").SAPGuiWindow(""SAP Eas..."
1,Set Transaktion: /n/MSG/R_V1,"SAPGuiSession(""Session"").SAPGuiWindow(""SAP Eas..."
2,Drück Enter,"SAPGuiSession(""Session"").SAPGuiWindow(""SAP Eas..."
3,Set Buchungskreis: HGV1,"SAPGuiSession(""Session"").SAPGuiWindow(""SAP Eas..."
4,Set Vertragstyp: 1,"SAPGuiSession(""Session"").SAPGuiWindow(""SAP Eas..."


In [20]:
# reading the first two script as sentences from the file
#sentences_x = [df['Skript'][0],df['Skript'][1]]
sentences_x = []
for i in df['Skript']:
    sentences_x.append(i)
type(sentences_x)

list

In [21]:
sentences_x

['SAPGuiSession("Session").SAPGuiWindow("SAP Easy Access  -  Benutzerme").Maximize',
 'SAPGuiSession("Session").SAPGuiWindow("SAP Easy Access  -  Benutzerme").SAPGuiOKCode("OKCode").Set "/n/msg/r_v1"',
 'SAPGuiSession("Session").SAPGuiWindow("SAP Easy Access  -  Benutzerme").SendKey ENTER',
 'SAPGuiSession("Session").SAPGuiWindow("SAP Easy Access  -  Benutzerme").SAPGuiEdit("Buchungskreis").Set "hgv1"',
 'SAPGuiSession("Session").SAPGuiWindow("SAP Easy Access  -  Benutzerme").SAPGuiEdit("Vertragstyp").Set "1"',
 'SAPGuiSession("Session").SAPGuiWindow("SAP Easy Access  -  Benutzerme").SAPGuiEdit("Vertragsartgruppe").Set "npfak"',
 'SAPGuiSession("Session").SAPGuiWindow("SAP Easy Access  -  Benutzerme").SAPGuiEdit("Vertragsartgruppe").SetFocus',
 'SAPGuiSession("Session").SAPGuiWindow("SAP Easy Access  -  Benutzerme").SendKey ENTER',
 'SAPGuiSession("Session").SAPGuiWindow("SAP Easy Access  -  Benutzerme").SAPGuiStatusBar("StatusBar").Sync',
 'SAPGuiSession("Session").SAPGuiWindow("SAP E

In [22]:
#``RegexpTokenizer`` splits a string into substrings using a regular expression
# token_s stores the individual words
# for now keeping the stopwords in the sentences
from nltk import RegexpTokenizer
from nltk.corpus import stopwords
tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+\(\"\".')
token_s=[]
for i in range(19):
    token_s += [tokenizer.tokenize(sentences_x[i])]


In [23]:
# check out how token_s looks like now
token_s

[['SAPGuiSession',
  'Session',
  'SAPGuiWindow',
  'SAP',
  'Easy',
  'Access',
  'Benutzerme',
  'Maximize'],
 ['SAPGuiSession',
  'Session',
  'SAPGuiWindow',
  'SAP',
  'Easy',
  'Access',
  'Benutzerme',
  'SAPGuiOKCode',
  'OKCode',
  'Set',
  'n',
  'msg',
  'r_v1'],
 ['SAPGuiSession',
  'Session',
  'SAPGuiWindow',
  'SAP',
  'Easy',
  'Access',
  'Benutzerme',
  'SendKey',
  'ENTER'],
 ['SAPGuiSession',
  'Session',
  'SAPGuiWindow',
  'SAP',
  'Easy',
  'Access',
  'Benutzerme',
  'SAPGuiEdit',
  'Buchungskreis',
  'Set',
  'hgv1'],
 ['SAPGuiSession',
  'Session',
  'SAPGuiWindow',
  'SAP',
  'Easy',
  'Access',
  'Benutzerme',
  'SAPGuiEdit',
  'Vertragstyp',
  'Set',
  '1'],
 ['SAPGuiSession',
  'Session',
  'SAPGuiWindow',
  'SAP',
  'Easy',
  'Access',
  'Benutzerme',
  'SAPGuiEdit',
  'Vertragsartgruppe',
  'Set',
  'npfak'],
 ['SAPGuiSession',
  'Session',
  'SAPGuiWindow',
  'SAP',
  'Easy',
  'Access',
  'Benutzerme',
  'SAPGuiEdit',
  'Vertragsartgruppe',
  'SetFocus

In [24]:
# assigning a list of labels for each skript, the labels are mimics of actual sentences
# need to test here if Test sentences can be put as labels
LabelList = []
max_rows = 19
for i in range(max_rows):
    LabelList.append(df['Beschreibung'][i])

In [25]:
LabelList

['Maximize SAPGuiSession',
 'Set Transaktion: /n/MSG/R_V1 ',
 'Drück Enter',
 'Set Buchungskreis: HGV1',
 'Set Vertragstyp: 1',
 'Set Vertragsartgruppe: NPFAK',
 'Bearbeite Vertragsartgruppe',
 'Press Enter',
 'Warte auf StatusBar',
 'Neuanlage Button klicken',
 'Set Verwendung RM: Ja',
 'Set Abr.über. aus RM: Ja',
 'Bezeichnung: Defect 16915',
 'Bezeichnung Zedent: AXA',
 'Set Partner1 (Verantw.) : cka',
 'Set Partner2 (Verantw.) : cka',
 'Set Partner3 (Verantw.) : cka',
 'SelectCell Partner',
 'Click Save']

In [26]:
# label the doc
from gensim.models.doc2vec import LabeledSentence, TaggedDocument 
max_rows = 19
tag_doc = []
for i in range(max_rows):
    #print (i)
    tag_doc.append(TaggedDocument(token_s[i],LabelList[i]))

In [27]:
# check words and tags pair in LabeledSentence
type(tag_doc)
tag_doc

[TaggedDocument(words=['SAPGuiSession', 'Session', 'SAPGuiWindow', 'SAP', 'Easy', 'Access', 'Benutzerme', 'Maximize'], tags='Maximize SAPGuiSession'),
 TaggedDocument(words=['SAPGuiSession', 'Session', 'SAPGuiWindow', 'SAP', 'Easy', 'Access', 'Benutzerme', 'SAPGuiOKCode', 'OKCode', 'Set', 'n', 'msg', 'r_v1'], tags='Set Transaktion: /n/MSG/R_V1 '),
 TaggedDocument(words=['SAPGuiSession', 'Session', 'SAPGuiWindow', 'SAP', 'Easy', 'Access', 'Benutzerme', 'SendKey', 'ENTER'], tags='Drück Enter'),
 TaggedDocument(words=['SAPGuiSession', 'Session', 'SAPGuiWindow', 'SAP', 'Easy', 'Access', 'Benutzerme', 'SAPGuiEdit', 'Buchungskreis', 'Set', 'hgv1'], tags='Set Buchungskreis: HGV1'),
 TaggedDocument(words=['SAPGuiSession', 'Session', 'SAPGuiWindow', 'SAP', 'Easy', 'Access', 'Benutzerme', 'SAPGuiEdit', 'Vertragstyp', 'Set', '1'], tags='Set Vertragstyp: 1'),
 TaggedDocument(words=['SAPGuiSession', 'Session', 'SAPGuiWindow', 'SAP', 'Easy', 'Access', 'Benutzerme', 'SAPGuiEdit', 'Vertragsartgruppe',

In [29]:
# if there are more than one sentence in TaggedDocument [ls], Doc2Vec model is failing 
from gensim.models.doc2vec import Doc2Vec
# initialize a model
model = Doc2Vec(vector_size=300, window=10, workers=1, alpha=0.025, min_alpha=0.01)

In [30]:
#build vocab
model.build_vocab(tag_doc)

In [31]:
 # get the initial document vector
docvec1 = model.docvecs[0]
vectors_docs = model.docvecs.vectors_docs[0]

In [32]:
vectors_docs, docvec1

(array([ 1.26084825e-03,  1.62353623e-03, -1.16876960e-04, -3.32758675e-04,
        -1.53312215e-03, -1.40082557e-03,  1.64607214e-03, -5.40794164e-04,
        -8.36672378e-04,  5.89121228e-05,  1.54547009e-03,  1.04503846e-03,
        -7.59160146e-04,  1.66317390e-04, -1.13735278e-03,  1.02489511e-03,
        -1.94127293e-04, -1.22064888e-03, -7.23884848e-04,  1.25357159e-03,
         9.42214741e-04,  1.11870223e-03,  9.78728713e-05,  1.33702275e-03,
         4.95025539e-04, -9.25829692e-04,  5.45195478e-04,  1.41082401e-03,
        -1.03395886e-03, -4.63464588e-04, -1.01485988e-03, -9.89180058e-04,
         3.84335435e-04,  3.25330795e-04, -6.47498237e-05, -1.31028355e-03,
        -1.93415704e-04, -1.22902833e-03,  3.63018306e-04, -1.70183717e-04,
        -4.61995311e-04,  9.66251770e-04, -1.18336131e-04,  1.14178169e-03,
        -1.25603541e-03,  1.49201485e-04,  1.74291737e-04, -9.92195564e-05,
        -6.74230250e-05, -3.20234947e-04, -2.61404755e-04,  7.71034684e-06,
         1.0

In [57]:
  # calculate most similar documents
    # (the model is not trained, so the results should be wrong)
#id_str = int(id_str)
test_doc2vec = model.docvecs[1]
docsim1 = model.docvecs.most_similar(positive = [test_doc2vec], topn=3)
#type(id_str)
#test_doc2vec

In [34]:
docsim1

[('a', 1.0), ('H', 0.10422752052545547), ('5', 0.09775698930025101)]

In [35]:
    # train this model
model.train(tag_doc, total_examples=len(tag_doc), epochs=10)

In [66]:
  # get the trained document vector
docvec2 = model.docvecs[0]
docvecsyn2 = model.docvecs.vectors_docs[0]

In [73]:
   # calculate most similar documents
    # (we expect the results to be correct)
#docsim2 = model.docvecs.most_similar(id_str)
test_doc2vec = model.docvecs['B']
#docsim2 = model.docvecs.most_similar(positive=[test_doc2vec])
docsim2 = model.docvecs.most_similar([docvec2])
print ('similar docs are')
print (docsim2)


similar docs are
[('M', 0.9961419105529785), ('e', 0.11882452666759491), ('E', 0.1102503314614296), ('b', 0.1046438217163086), ('T', 0.09696125984191895), ('u', 0.08850930631160736), ('J', 0.07386977225542068), (')', 0.07157612591981888), ('h', 0.0689878910779953), (' ', 0.06716625392436981)]


In [64]:
word_sim = model.wv.similar_by_word('SAPGuiWindow')
word_sim

[('Access', 0.08042235672473907),
 ('Easy', 0.033343322575092316),
 ('Session', 0.026062119752168655),
 ('SAPGuiSession', -0.020474687218666077),
 ('Set', -0.024982942268252373),
 ('SAPGuiEdit', -0.0342138335108757),
 ('SAP', -0.06253444403409958),
 ('Benutzerme', -0.11362167447805405)]

In [41]:
# information about system, python and module version
import platform; print(platform.platform())
import sys; print("Python", sys.version)
import numpy; print("NumPy", numpy.__version__)
import scipy; print("SciPy", scipy.__version__)
import gensim; print("gensim", gensim.__version__)
from gensim.models import word2vec;print("FAST_VERSION", word2vec.FAST_VERSION)

Windows-7-6.1.7601-SP1
Python 3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]
NumPy 1.14.0
SciPy 1.0.0
gensim 3.4.0
FAST_VERSION 1


In [68]:
  # choose one document
doc = tag_doc[0].words

In [70]:
    # infer vectors with different 'steps' parameters
infervec1 = model.infer_vector(doc, alpha=0.025, min_alpha=0.01, steps=1)
infervec2 = model.infer_vector(doc, alpha=0.025, min_alpha=0.01, steps=10)
infervec3 = model.infer_vector(doc, alpha=0.025, min_alpha=0.01, steps=100)
infervec4 = model.infer_vector(doc, alpha=0.025, min_alpha=0.01, steps=1000)

In [73]:
   # print results

    # document vector
print('Document vector:')

    # we can see that the document vectors do not change after training.
print(docvec1[:5])
print(docvec2[:5])
print(vectors_docs[:5])
print(docvecsyn2[:5])

    # most similar documents
print('Most similar:')

    # before training, the result is wrong. after training, correct. good.
print(docsim1[:2])
print(docsim2[:2])

    # infered vectors with different 'steps' parameters
print('Infered vector:')

    # we can see that, they are quite different.
print(infervec1[:5])
print(infervec2[:5])
print(infervec3[:5])
print(infervec4[:5])

    # norm of inferred vectors
print('Norm of infered vector:')

    # it seems that, the norm of inferred vectors would be larger for bigger steps
print(np.linalg.norm(infervec1))
print(np.linalg.norm(infervec2))
print(np.linalg.norm(infervec3))
print(np.linalg.norm(infervec4))


Document vector:
[ 2.9753798e-03 -2.4279458e-03  9.7049524e-05 -2.8524832e-03
  2.7555079e-04]
[ 2.9753798e-03 -2.4279458e-03  9.7049524e-05 -2.8524832e-03
  2.7555079e-04]
[ 2.9753798e-03 -2.4279458e-03  9.7049524e-05 -2.8524832e-03
  2.7555079e-04]
[ 2.9753798e-03 -2.4279458e-03  9.7049524e-05 -2.8524832e-03
  2.7555079e-04]
Most similar:
[('Neuanlage klicken', -0.03615549951791763)]
[('Maximize', -0.03615549951791763)]
Infered vector:
[ 0.00382353 -0.00492315 -0.0034303  -0.00365737  0.00150728]
[ 0.00385699 -0.00490817 -0.00344525 -0.00367717  0.00149982]
[ 0.00408777 -0.00480479 -0.00354841 -0.00381376  0.00144835]
[ 0.00613369 -0.00399387 -0.00442289 -0.00505528  0.00101849]
Norm of infered vector:
0.031142825
0.031136705
0.031138709
0.034457378


In [70]:
from IPython.display import HTML

In [77]:
# footnote at the end
HTML('''<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

<footer id="attribution" style="float:right; color:#999; background:#fff;">
Machine Learning Use Case - Saurabh Kumar.
</footer>''')

In [76]:
HTML('''<script>
  jQuery(document).ready(function($) {

  $(window).load(function(){
    $('#preloader').fadeOut('slow',function(){$(this).remove();});
  });

  });
</script>

<style type="text/css">
  div#preloader { position: fixed;
      left: 0;
      top: 0;
      z-index: 999;
      width: 100%;
      height: 100%;
      overflow: visible;
      background: #fff url('http://preloaders.net/preloaders/720/Moving%20line.gif') no-repeat center center;
  }

</style>

<div id="preloader"></div>''')

In [78]:
# hide the code above
HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>''')
